In [2]:
import qiskit

from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.fake_backends import Grid, Hexagonal13, Hexagonal16, FullyConnected, SquareOctagonal, Hexagonal
from PatchedMeasCal.inv_measure_methods import aim, sim
from PatchedMeasCal import jigsaw

from PatchedMeasCal import state_prep_circuits
from PatchedMeasCal.bv import bv_circuit_cmap
from PatchedMeasCal.fake_measurement_distributions import renormalise_measurement_results
from PatchedMeasCal.utils import Progressbar
from PatchedMeasCal.qiskit_meas_fitters import qiskit_full, qiskit_linear

from PatchedMeasCal import non_standard_coupling_maps

from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import tensored_meas_cal, TensoredMeasFitter

from functools import partial

import qiskit.tools.jupyter

import random
import mthree

In [3]:
def distance_measure(x, bv_value):
    if bv_value in x:
        return 1 - (x[bv_value] / sum(x.values()))
    return 1

In [5]:
n_shots = 8000
n_rounds = 5

circuit_res = []
aim_res = []
sim_res = []
full_res = []
lin_res = []
tpf_res = []
tpf_e_res = []
jr_res = []
m3_res = []

backend_generators = [
    partial(Hexagonal, 1, 1),
    partial(Hexagonal, 2, 1),
    Hexagonal13,
    partial(Hexagonal, 3, 1),
    Hexagonal16
]

pb = Progressbar(20, 5 * n_rounds * len(backend_generators), 'Sims')

for backend_gen in backend_generators:
    
    circuit_round = []
    aim_round = []
    sim_round = []
    tpf_round = []
    tpf_e_round = []
    jr_round = []
    m3_round = []
#     full_round = []
#     linear_round = []
    
    for r_num in range(n_rounds):
            
        backend = backend_gen()
        n_qubits = len(backend.properties().qubits)
        
        err_cmap = non_standard_coupling_maps.error_coupling_map(backend=backend)
        
        # Build tensor patches
        tpf = TensorPatchFitter(backend, n_shots=n_shots // 2)
        tpf.build()
        
        tpf_e = TensorPatchFitter(backend, n_shots=n_shots // 2, coupling_map=err_cmap)
        tpf_e.build()
        
        # M3
        mit = mthree.M3Mitigation(backend)
        mit.cals_from_system(list(range(n_qubits)), n_shots // 2)
        
#         full = qiskit_full(backend, n_qubits, n_shots)
#         linear = qiskit_linear(backend, n_qubits, n_shots)

        # Run Circuits
        target_value = random.randint(0, 2 ** (n_qubits - 1) - 1)
        bv_value = bin((target_value << 1) + 1)[2:].zfill(n_qubits)[::-1]
        bv_string = bin(target_value)[2:].zfill(n_qubits - 1)
        circuit = bv_circuit_cmap(bv_string, n_qubits, backend)
        dist_measure = lambda x: distance_measure(x, bv_value)
           
        pb.tick('Bare: {}'.format(n_qubits))
        
        # Transpile and run circuit
        tc = qiskit.transpile(circuit,
                              backend,
                              optimization_level=0,
                              initial_layout=list(range(n_qubits))
                             )
        cr = qiskit.execute(circuit, 
                             backend, 
                             shots=n_shots / 2, 
                             optimization_level=0,
                             initial_layout=list(range(n_qubits))
                            ).result().get_counts()
        circuit_round.append(dist_measure(cr))
        
        m3_round.append(dist_measure(
            mit.apply_correction(cr, list(range(n_qubits))).nearest_probability_distribution()
        ))
        
        # Apply AIM to circuit
        pb.tick('AIM {}'.format(n_qubits))
        ar = aim(circuit, backend, n_qubits, n_shots=n_shots, equal_shot_distribution=True)
        aim_round.append(dist_measure(ar))

        # Apply SIM to circuit
        pb.tick('SIM {}'.format(n_qubits))
        sr = sim(circuit, backend, n_qubits, n_shots=n_shots, equal_shot_distribution=True) 
        sr = renormalise_measurement_results(sr, 1)
        sim_round.append(dist_measure(sr))

        ### DO NOT RUN FOR LARGE NUMBERS OF QUBITS (>8-10)
        ### WILL TAKE FOREVER, THEN OOM
#         # Apply Full
#         pb.tick('Full {}'.format(n_qubits))
#         fr = full.apply(cr)
#         full_round.append(dist_measure(fr))

#         # Apply Linear
#         pb.tick('Lin {}'.format(n_qubits))
#         lr = linear.apply(cr)
#         linear_round.append(dist_measure(lr))

        # Apply CMC to circuit
        pb.tick('CMC {}'.format(n_qubits))
        tr = tpf.apply(cr)
        tpf_round.append(dist_measure(tr))
        
        # Apply CMC to circuit
        pb.tick('CMC E{}'.format(n_qubits))
        tr = tpf_e.apply(cr)
        tpf_e_round.append(dist_measure(tr))

        # Jigsaw
        pb.tick('JIG {}'.format(n_qubits))
        jr = jigsaw.jigsaw(circuit, backend, n_shots, equal_shot_distribution=True)
        jr_round.append(dist_measure(jr))
        #print(jr_round)
        
    circuit_res.append(circuit_round)
    aim_res.append(aim_round)
    sim_res.append(sim_round)
#     full_res.append(full_round)
#     lin_res.append(linear_round)
    tpf_res.append(tpf_round)
    tpf_e_res.append(tpf_e_round)
    jr_res.append(jr_round)
    m3_res.append(m3_round)

results = {
'bare':circuit_res,
'aim':aim_res,
'sim':sim_res,
'full':full_res,
'lin':lin_res,
'cmc':tpf_res,
'cmc_err':tpf_e_res,
'jig':jr_res,
'm3':m3_res
}

Sims : [====================] 100.0% CMC E16 ETA: 0s
Sims : [====================] 100.0% JIG 16 ETA: 0s
Sims : [====================] 100.0% Bare: 16 ETA: 0s
Sims : [====================] 100.0% AIM 16 ETA: 0s
Sims : [====================] 100.0% SIM 16 ETA: 0s
Sims : [====================] 100.0% CMC 16 ETA: 0s
Sims : [====================] 100.0% CMC E16 ETA: 0s
Sims : [====================] 100.0% JIG 16 ETA: 0s
Sims : [====================] 100.0% Bare: 16 ETA: 0s
Sims : [====================] 100.0% AIM 16 ETA: 0s
Sims : [====================] 100.0% SIM 16 ETA: 0s
Sims : [====================] 100.0% CMC 16 ETA: 0s
Sims : [====================] 100.0% CMC E16 ETA: 0s
Sims : [====================] 100.0% JIG 16 ETA: 0s
Sims : [====================] 100.0% Bare: 16 ETA: 0s
Sims : [====================] 100.0% AIM 16 ETA: 0s
Sims : [====================] 100.0% SIM 16 ETA: 0s
Sims : [====================] 100.0% CMC 16 ETA: 0s
Sims : [====================] 100.0% CMC E16 ETA: 0s
Si

In [6]:
results

{'bare': [[0.31575, 0.3235, 0.32325000000000004, 0.30825, 0.27725],
  [0.45999999999999996,
   0.47224999999999995,
   0.476,
   0.4475,
   0.47824999999999995],
  [0.552, 0.60775, 0.56775, 0.55375, 0.628],
  [0.59825, 0.5985, 0.59875, 0.59425, 0.6012500000000001],
  [0.683,
   0.6545000000000001,
   0.6777500000000001,
   0.63225,
   0.6467499999999999]],
 'aim': [[0.29566666666666663,
   0.33099999999999996,
   0.2826666666666666,
   0.31433333333333335,
   0.2686666666666667],
  [0.4731666666666666,
   0.45466666666666666,
   0.4711666666666666,
   0.44733333333333336,
   0.476],
  [0.5429999999999999,
   0.5848333333333333,
   0.5688333333333333,
   0.5446666666666666,
   0.6285000000000001],
  [0.5886666666666667,
   0.5898333333333333,
   0.5951666666666666,
   0.5981666666666667,
   0.5823333333333334],
  [0.6723333333333333,
   0.6475,
   0.6646666666666667,
   0.6291666666666667,
   0.6268333333333334]],
 'sim': [[0.2971250007818599,
   0.34687499489518847,
   0.28199999442356

In [ ]:
grid_4_6_8 = {'bare': [[0.21199999999999997,
   0.17800000000000005,
   0.21175,
   0.21799999999999997,
   0.21475],
  [0.3325, 0.2835, 0.33875, 0.32925000000000004, 0.32625000000000004],
  [0.4135, 0.39425, 0.42800000000000005, 0.40900000000000003, 0.45075]],
 'aim': [[0.1995,
   0.20350000000000001,
   0.20299999999999996,
   0.20966666666666667,
   0.19699999999999995],
  [0.3461666666666666,
   0.2968333333333333,
   0.32066666666666666,
   0.31833333333333336,
   0.30266666666666664],
  [0.42200000000000004,
   0.40149999999999997,
   0.39749999999999996,
   0.4,
   0.4298333333333333]],
 'sim': [[0.22699999395500392,
   0.2253749932159813,
   0.23037500475081463,
   0.2211249943117003,
   0.2123750028457616],
  [0.331124996757144,
   0.31037500074447055,
   0.3273750107972935,
   0.32612500002488565,
   0.31525000373944556],
  [0.4248749956646809,
   0.4067499972820189,
   0.3882500087338737,
   0.37850000779440574,
   0.4351250076682025]],
 'full': [[0.03193818580704444,
   0.03568881982781169,
   0.06632158416375245,
   0.04503818652778935,
   0.020335073062870967],
  [0.055682654332176784,
   0.03399700908596337,
   0.07255046011297106,
   0.07801087490135794,
   0.053807712892897874],
  [0.09897005900366618,
   0.11650198740593287,
   0.10930120054963333,
   0.07987240060671008,
   0.10183964421768499]],
 'lin': [[0.03860455971857757,
   0.03130462344349094,
   0.06248380079143301,
   0.049861012796931115,
   0.0246297839003371],
  [0.067763978441017,
   0.03146532274271463,
   0.09352497077977817,
   0.07663858216436115,
   0.06715344099371279],
  [0.09259618962393246,
   0.12558488376462795,
   0.11274673897265741,
   0.10134853125191656,
   0.08308233066217141]],
 'cmc': [[0.08884985222952824,
   0.03874695641643766,
   0.050581583986626266,
   0.033628818157068974,
   0.08499626382105174],
  [0.08589094292325872,
   0.08618792807337483,
   0.10910106798380315,
   0.12673810357034276,
   0.11996021124179701],
  [0.14032347339348672,
   0.15896839166116783,
   0.18622731505780632,
   0.1965149585969782,
   0.16124657875779969]],
 'cmc_err': [[0.05388565619623942,
   0.041748962000427214,
   0.040536190602102895,
   0.04702827885127914,
   0.04849335851077807],
  [0.10620028357062938,
   0.08209710755251387,
   0.11652773756150248,
   0.10287339210535873,
   0.09251872723065402],
  [0.12457800194938462,
   0.1343587801855527,
   0.16691510342391325,
   0.16298225331242877,
   0.18259238010286605]],
 'jig': [[0.03091610509639353,
   0.023531826308737513,
   0.027972539123753526,
   0.030084319713406638,
   0.03461658021146663],
  [0.08075924008084456,
   0.05083977455637423,
   0.053156768693155665,
   0.05024940596480565,
   0.06473364425909434],
  [0.08359428384337142,
   0.07735303713518193,
   0.07788127936820843,
   0.07079949150154974,
   0.11061272411848322]]}

In [8]:
grid_9_16 = {'bare': [[0.42674999999999996,
   0.45575,
   0.48650000000000004,
   0.469,
   0.46699999999999997],
  [0.43000000000000005,
   0.45825000000000005,
   0.48175,
   0.474,
   0.46725000000000005],
  [0.5525, 0.5125, 0.502, 0.5387500000000001, 0.5297499999999999],
  [0.5802499999999999, 0.64, 0.54725, 0.59775, 0.5855],
  [0.5795, 0.60375, 0.62575, 0.6214999999999999, 0.5842499999999999],
  [0.6025, 0.5740000000000001, 0.6327499999999999, 0.68625, 0.62025]],
 'aim': [[0.4136666666666666,
   0.4201666666666667,
   0.4628333333333333,
   0.4431666666666667,
   0.4651666666666666],
  [0.43033333333333335,
   0.4525,
   0.48250000000000004,
   0.4701666666666666,
   0.4741666666666666],
  [0.5588333333333333,
   0.5086666666666666,
   0.48883333333333334,
   0.523,
   0.5191666666666667],
  [0.5883333333333334,
   0.6203333333333334,
   0.5298333333333334,
   0.5723333333333334,
   0.5913333333333333],
  [0.5815, 0.5916666666666667, 0.6148333333333333, 0.6453333333333333, 0.611],
  [0.6325000000000001,
   0.5945,
   0.6346666666666667,
   0.6803333333333333,
   0.6143333333333334]],
 'sim': [[0.4181249999466763,
   0.42587499070076473,
   0.4687500107688718,
   0.46075000607335803,
   0.4612499858063539],
  [0.4442500063241639,
   0.461249985453561,
   0.48962499905097867,
   0.467624996052886,
   0.46774998784478605],
  [0.5463749988345208,
   0.5207500042832143,
   0.5078749938261975,
   0.5256250080246534,
   0.5373749963477605],
  [0.5992500045456943,
   0.6128750004415251,
   0.5446250039718177,
   0.5806249942870458,
   0.5902500036246527],
  [0.574125006358259,
   0.6062500029422153,
   0.6271250060297859,
   0.6402500094023087,
   0.6253749936723129],
  [0.6213750081749656,
   0.61900000317613,
   0.6301250008225869,
   0.6623749932540927,
   0.6286249944501526]],
 'full': [],
 'lin': [],
 'cmc': [[0.17390816671656228,
   0.21358095241320896,
   0.16409298982498488,
   0.17318283647503363,
   0.1927203475974525],
  [0.19844651263634772,
   0.200102899521443,
   0.21048112932801621,
   0.16583219088256362,
   0.1919037236970682],
  [0.2886967222133706,
   0.2128100405844654,
   0.21060325542956004,
   0.26308744937187156,
   0.22997924627074473],
  [0.3000845193267444,
   0.35626673576839496,
   0.25474617699303215,
   0.3050576686637886,
   0.28283806520360744],
  [0.27926078110848196,
   0.2904894589044321,
   0.3086352369165076,
   0.3457935735976376,
   0.27681780169225834],
  [0.3589365168076192,
   0.26851857225302134,
   0.3461695783547527,
   0.42347990223360477,
   0.34977031399728353]],
 'cmc_err': [[0.12832723859757078,
   0.23530993396026922,
   0.1821949400223266,
   0.1857224929587422,
   0.1480711817611875],
  [0.1689728052985937,
   0.17928255916689018,
   0.2366054806306438,
   0.183552529881727,
   0.19442327273658566],
  [0.2581465978392088,
   0.23244066175777978,
   0.20987572167855562,
   0.25480750547360564,
   0.19796185485044038],
  [0.27290172746545904,
   0.3861210495092663,
   0.24950101849149187,
   0.29569000567355974,
   0.29148443694633563],
  [0.26606071713255086,
   0.29580214721346465,
   0.3119533641190101,
   0.3297944333792977,
   0.2529526955075788],
  [0.3206711712015531,
   0.19915239816002372,
   0.32840445966047715,
   0.43677219132034084,
   0.27797726422350644]],
 'jig': [[0.12536152574504023,
   0.14244448347308625,
   0.14825030422211183,
   0.14554898202317368,
   0.17526509645731325],
  [0.15639194427465153,
   0.09905064828164112,
   0.10235315688416458,
   0.10797053481688568,
   0.09071847352090234],
  [0.26928846758416225,
   0.114492090107841,
   0.11752006018270744,
   0.1328881002746094,
   0.14648475553437879],
  [0.14248936839023751,
   0.17870271110528635,
   0.15042068046360946,
   0.1532313803969838,
   0.16947826866719606],
  [0.1905469606450736,
   0.2005340878762535,
   0.3430643957726619,
   0.25083204492999156,
   0.32413561261519885],
  [0.19010079965494708,
   0.2915843929058983,
   0.2248209214391914,
   0.21420451464738022,
   0.2034501656608413]]}